# Tesla & GameStop: Stock vs Revenue Dashboard

_Use yfinance and web scraping to build side-by-side price vs revenue dashboards for TSLA and GME._

## Environment & Install
Uncomment the next cell if you need to install packages in your environment.

In [1]:
pip install yfinance pandas plotly beautifulsoup4 lxml requests

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 25.2 -> 25.3
[notice] To update, run: python.exe -m pip install --upgrade pip


In [1]:
import pandas as pd
import numpy as np
import yfinance as yf
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import requests
from bs4 import BeautifulSoup

pd.set_option('display.max_columns', 50)

In [5]:
def get_quarterly_revenue_from_macrotrends(ticker, company_slug):
    """Scrape the quarterly revenue table from Macrotrends for a given ticker.
    
    Parameters
    ----------
    ticker : str
        Stock ticker (e.g., 'TSLA')
    company_slug : str
        Lowercase company name used in Macrotrends URL (e.g., 'tesla', 'gamestop')
    Returns
    -------
    DataFrame with columns ['Date', 'Revenue'] sorted by Date ascending.
    """
    url = f"https://www.macrotrends.net/stocks/charts/{ticker}/{company_slug}/revenue"
    # Add headers to mimic a browser request with updated User-Agent
    headers = {
        'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/131.0.0.0 Safari/537.36',
        'Accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,image/avif,image/webp,image/apng,*/*;q=0.8',
        'Accept-Language': 'en-US,en;q=0.9',
        'Accept-Encoding': 'gzip, deflate, br',
        'Connection': 'keep-alive',
        'Upgrade-Insecure-Requests': '1',
        'Sec-Fetch-Dest': 'document',
        'Sec-Fetch-Mode': 'navigate',
        'Sec-Fetch-Site': 'none',
        'Cache-Control': 'max-age=0',
    }
    # Fetch the page with proper headers
    response = requests.get(url, headers=headers)
    response.raise_for_status()
    
    # Pull all HTML tables from the response
    tables = pd.read_html(response.text, flavor='lxml')
    # Heuristic: quarterly revenue table usually has 'Quarterly' in the first column header or a 'Date' and 'Revenue' pair
    qtables = [t for t in tables if t.columns.str.contains('Quarter', case=False).any() or set(['Date','Revenue']).issubset(set(t.columns))]
    if not qtables:
        # fallback: try the first table
        df = tables[0].copy()
    else:
        # choose the widest table (often the quarterly one)
        df = max(qtables, key=lambda t: t.shape[0]).copy()
    # Normalize columns
    cols = [c.strip().replace('\n',' ') for c in df.columns]
    df.columns = cols
    # Rename common variants
    if 'Quarter' in df.columns: df = df.rename(columns={'Quarter':'Date'})
    if 'Revenue (TTM)' in df.columns and 'Revenue' not in df.columns:
        df = df.rename(columns={'Revenue (TTM)':'Revenue'})
    if 'Revenue' not in df.columns:
        # Try to find the first revenue-like column
        rev_col = next((c for c in df.columns if 'Revenue' in c), None)
        if rev_col: df = df.rename(columns={rev_col:'Revenue'})
    # Keep Date + Revenue
    keep = [c for c in ['Date','Revenue'] if c in df.columns]
    df = df[keep].copy()
    # Clean Revenue: remove $ and commas; coerce to float (Millions of USD on Macrotrends)
    df['Revenue'] = (df['Revenue'].astype(str)
                     .str.replace(r'\$|,', '', regex=True)
                     .replace({'': np.nan, 'None': np.nan})).astype(float)
    # Drop missing and sort
    df = df.dropna(subset=['Revenue']).drop_duplicates(subset=['Date']).sort_values('Date')
    df.reset_index(drop=True, inplace=True)
    return df

In [2]:
def get_quarterly_revenue_from_yfinance(ticker):
    """Get quarterly revenue data from Yahoo Finance for a given ticker.
    
    Parameters
    ----------
    ticker : str
        Stock ticker (e.g., 'TSLA', 'GME')
    
    Returns
    -------
    DataFrame with columns ['Date', 'Revenue'] sorted by Date ascending.
    """
    stock = yf.Ticker(ticker)
    # Get quarterly financials (revenue is in the 'Total Revenue' row)
    financials = stock.quarterly_financials
    
    if financials.empty or 'Total Revenue' not in financials.index:
        print(f"Warning: No revenue data found for {ticker}")
        return pd.DataFrame(columns=['Date', 'Revenue'])
    
    # Extract Total Revenue row and transpose to get dates as rows
    revenue_data = financials.loc['Total Revenue'].sort_index()
    
    # Create DataFrame with proper format
    df = pd.DataFrame({
        'Date': revenue_data.index.strftime('%Y-%m-%d'),
        'Revenue': revenue_data.values / 1_000_000  # Convert to millions
    })
    
    df = df.sort_values('Date').reset_index(drop=True)
    return df

In [3]:
def make_graph(stock_data, revenue_data, title):
    """Create a two-panel Plotly figure with Close price (left) and Revenue (right)."""
    fig = make_subplots(rows=1, cols=2, subplot_titles=(f"{title} — Close Price", f"{title} — Quarterly Revenue (Millions USD)"))
    
    # Left: closing prices
    fig.add_trace(go.Scatter(x=stock_data.index, y=stock_data['Close'], name='Close', mode='lines'), row=1, col=1)
    
    # Right: revenue bar
    fig.add_trace(go.Bar(x=pd.to_datetime(revenue_data['Date']), y=revenue_data['Revenue'], name='Revenue (M)'), row=1, col=2)
    
    fig.update_xaxes(title_text='Date', row=1, col=1)
    fig.update_yaxes(title_text='Price (USD)', row=1, col=1)
    fig.update_xaxes(title_text='Quarter', row=1, col=2)
    fig.update_yaxes(title_text='Revenue (M USD)', row=1, col=2)
    fig.update_layout(title_text=title, showlegend=False, height=500, width=1000)
    return fig

## Question 1 — Extract Tesla Stock Data Using `yfinance`

In [4]:
tesla_ticker = yf.Ticker('TSLA')
tesla_data = tesla_ticker.history(period='max')
tesla_data.reset_index(inplace=True)
tesla_data.head()

,Date,Open,High,Low,Close,Volume,Dividends,Stock Splits
0,2010-06-29 00:00:00-04:00,1.266667,1.666667,1.169333,1.592667,281494500,0.0,0.0
1,2010-06-30 00:00:00-04:00,1.719333,2.028000,1.553333,1.588667,257806500,0.0,0.0
2,2010-07-01 00:00:00-04:00,1.666667,1.728000,1.351333,1.464000,123282000,0.0,0.0
3,2010-07-02 00:00:00-04:00,1.533333,1.540000,1.247333,1.280000,77097000,0.0,0.0
4,2010-07-06 00:00:00-04:00,1.333333,1.333333,1.055333,1.074000,103003500,0.0,0.0


## Question 2 — Extract Tesla Revenue via Web Scraping

In [5]:
tesla_revenue = get_quarterly_revenue_from_yfinance('TSLA')
tesla_revenue.tail()

,Date,Revenue
1,2024-06-30,25500.0
2,2024-09-30,25182.0
3,2024-12-31,25707.0
4,2025-03-31,19335.0
5,2025-06-30,22496.0


## Question 3 — Extract GameStop Stock Data Using `yfinance`

In [9]:
gme_ticker = yf.Ticker('GME')
gme_data = gme_ticker.history(period='max')
gme_data.reset_index(inplace=True)
gme_data.head()

,Date,Open,High,Low,Close,Volume,Dividends,Stock Splits
0,2002-02-13 00:00:00-05:00,1.620128,1.693350,1.603296,1.691666,76216000,0.0,0.0
1,2002-02-14 00:00:00-05:00,1.712707,1.716074,1.670626,1.683251,11021600,0.0,0.0
2,2002-02-15 00:00:00-05:00,1.683250,1.687458,1.658002,1.674834,8389600,0.0,0.0
3,2002-02-19 00:00:00-05:00,1.666418,1.666418,1.578047,1.607504,7410400,0.0,0.0
4,2002-02-20 00:00:00-05:00,1.615921,1.662210,1.603296,1.662210,6892800,0.0,0.0


## Question 4 — Extract GameStop Revenue via Web Scraping

In [10]:
gme_revenue = get_quarterly_revenue_from_yfinance('GME')
gme_revenue.tail()

,Date,Revenue
1,2024-07-31,798.3
2,2024-10-31,860.3
3,2025-01-31,1282.6
4,2025-04-30,732.4
5,2025-07-31,972.2


## Question 5 — Tesla Stock and Revenue Dashboard

In [6]:
fig_tsla = make_graph(tesla_data.set_index('Date'), tesla_revenue, 'Tesla (TSLA)')
fig_tsla.show()

## Question 6 — GameStop Stock and Revenue Dashboard

In [11]:
fig_gme = make_graph(gme_data.set_index('Date'), gme_revenue, 'GameStop (GME)')
fig_gme.show()

## Question 7 — Share your Assignment Notebook
1. Save the notebook as `tsla_gme_dashboard.ipynb`.
2. Push it to a **public GitHub repo** (Add file → Upload files → Commit).
3. Paste the GitHub link in your assignment submission.
